# This is  a research notebook to try to find correlation between hyperwave and volume 

In [ ]:
# !pip install --upgrade pip

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')

/home/nbuser/anaconda3_501/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nbuser/anaconda3_501/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## This is the place where you setup the symbol and phases

In [2]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'symbol':'DDD',
    'price_range_size':5,
    'phases':{
        "phase1":(datetime(2000, 1, 1), datetime(2012, 5, 14)),
        "phase2":(datetime(2012, 5, 14), datetime(2013, 3, 18)),
        "phase3":(datetime(2013, 3, 18), datetime(2013, 10, 7)),
        "phase4":(datetime(2013, 10, 7), datetime(2014, 1, 14)),
        "phase5":(datetime(2014, 1, 14), datetime(2014, 5, 5)),
        "phase6":(datetime(2014, 5, 5), datetime(2014, 7, 7)),
        "phase7":(datetime(2014, 7, 7) , datetime(2015, 7, 6))
    }
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'symbol':'VRX',
    'price_range_size':10,
    'phases':{
        "phase1":(datetime(2008, 3, 10), datetime(2010, 6, 21)),
        "phase2":(datetime(2010, 6, 21), datetime(2012, 7, 9)),
        "phase3":(datetime(2012, 7, 9), datetime(2014, 10, 27)),
        "phase4":(datetime(2014, 10, 27), datetime(2015, 8, 10)),
        "phase5":(datetime(2015, 8, 10), datetime(2015, 11, 9)),
        "phase6":(datetime(2015, 11, 9), datetime(2015, 12, 28)),
        "phase7":(datetime(2015, 12, 28), datetime(2016, 10, 31))
    }
}


In [3]:
company_setup = data_Valeant
name = company_setup ['name'] 
symbol = company_setup ['symbol']
price_range_size = company_setup ['price_range_size']
phases = company_setup['phases']

# constant used for other computation
root_date = datetime(1800, 1, 1)

## Fetch the data

In [7]:
def get_raw_historical_data(symbol, timeframe='daily'):
    url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(symbol, timeframe)
    print("Url to fetch the data {}".format(url_symbol))
    print("fetching data for {} on {}...".format(symbol, timeframe))
    df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
    df_price = df_list[0].dropna()
    return df_price

# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price_raw = get_raw_historical_data(symbol, 'Weekly')

Url to fetch the data https://www.investopedia.com/markets/api/partial/historical/?Symbol=VRX&Type=Historical+Prices&Timeframe=Weekly&StartDate=Jan+01%2C+1900
fetching data for VRX on Weekly...


In [24]:
def get_nb_weeks(row, base_date = root_date):
    return int((row["Date"]-base_date).days/7)

def clean_raw_market_data(df):
    df.loc[:,('Date')] = pd.to_datetime(df.loc[:,('Date')])
    df = df.rename(columns={'Adj. Close':'Close'})
    df['IsPriceClosingUp'] = df.Close > df.Close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row),axis=1)
    return df

# df_daily_price = clean_raw_market_data(df_daily_price_raw)
df_weekly_price = clean_raw_market_data(df_weekly_price_raw)

In [25]:
print(df_weekly_price)

           Date   Open   High    Low  Close      Volume  IsPriceClosingUp  \
0    2018-07-13  24.15  24.43  23.36  23.40   6562058.0             False   
1    2018-07-06  22.94  23.06  22.76  22.96   4630733.0             False   
2    2018-06-29  23.71  24.06  23.08  23.24   4967458.0              True   
3    2018-06-22  24.29  24.47  23.82  24.27   6174842.0              True   
4    2018-06-15  26.76  27.11  26.63  26.86   6070686.0              True   
5    2018-06-08  25.00  26.09  24.94  25.79  10602499.0             False   
6    2018-06-01  22.13  22.24  22.05  22.08   3990433.0             False   
7    2018-05-25  22.16  22.32  22.09  22.24   3470573.0              True   
8    2018-05-18  22.06  22.24  21.97  22.14   5488345.0             False   
9    2018-05-11  20.07  20.93  19.82  20.85  10885417.0             False   
10   2018-05-04  17.62  18.05  17.44  18.02   5478651.0             False   
11   2018-04-27  17.85  18.25  17.82  18.07   5664572.0              True   

In [40]:
def get_weekId_cartesian_product(df):
    df_from = df[['weekId']].rename(index=str, columns={"weekId": "weekId_from"})
    df_to = df[['weekId']].rename(index=str, columns={"weekId": "weekId_to"})
    df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

    return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]

In [46]:
def get_weekId_max_price(df):
    return df.ix[ df['Close'].argmax()]["weekId"]